In [1]:
import sys,os
import numpy as np
import pandas as pd

import cclib
from cclib.bridge.cclib2ase import makease
from cclib.bridge.cclib2openbabel import makeopenbabel

from ase.geometry.analysis import Analysis

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.molecule_matcher import MoleculeMatcher, HungarianOrderMatcher
from pymatgen.core.composition import Composition
from pymatgen.core import Molecule
from pymatgen.symmetry.analyzer import PointGroupAnalyzer

from IPython.display import display, Markdown, Latex




In [2]:
#Answer data
filename1 = "C:/Users/eenadbu/PortableApps/Git/Notebook/CH4_opt_631g_d/output.out"
data1 = cclib.io.ccread(filename1)
filename2 = "C:/Users/eenadbu/PortableApps/Git/Notebook/CH4_opt_63111++g_dd/output.out"
data2 = cclib.io.ccread(filename2)
filename3 = "C:/Users/eenadbu/PortableApps/Git/Notebook/CH4_opt_ccp-VTZ/output.out"
data3 = cclib.io.ccread(filename3)

#Student info

#User = 'eenadbu'

JobID1 = 18356
JobID2 = 18746
JobID3 = 18777


#Student data
#filename4 = "C:/Users/eenadbu/PortableApps/Git/Notebook/hand_clean/output.out"
#filename5 = "C:/Users/eenadbu/PortableApps/Git/Notebook/hand_rubbish/output.out"
#filename6 = "C:/Users/eenadbu/PortableApps/Git/Notebook/CH4_opt_ccp-VTZ/output.out"




In [3]:
data4 = cclib.io.ccread(filename4)
data5 = cclib.io.ccread(filename5)
data6 = cclib.io.ccread(filename6)

In [4]:
filelist = [data1, data2, data3, data4, data5, data6]
runlist = ["run 1", "run 2", "run 3"]
anslist = ["answer 1", "answer 2", "answer 3"]


def ProcessFiles(filelist):

    data = {"Basis Set":[], "Functional":[], "Success":[], "Package":[], "Initial Atom Coords":[], "Final Atom Coords":[],
           "Atom Nos":[], "Charge":[], "Multiplicity":[], "# Electrons": [], "# atoms": [], "Opt Done": [], "SCF Energies":[],
           "Ase Initial":[], "Ase Final":[], "Pymat Initial":[], "Pymat Final":[], "Reduced Formula":[], "Initial Distance Matrix":[],
           "Final Distance Matrix":[], "Initial Symm":[], "Final Symm Low Tol": [], "Final Symm High Tol": []}

    for i in filelist:
        data["Basis Set"].append(i.metadata["basis_set"])
        data["Functional"].append(i.metadata["functional"])
        data["Success"].append(i.metadata["success"])
        data["Package"].append(i.metadata["package"])
        data["Initial Atom Coords"].append(i.atomcoords[0])
        data["Final Atom Coords"].append(i.atomcoords[-1])
        data["Atom Nos"].append(i.atomnos)
        data["Charge"].append(i.charge)
        data["Multiplicity"].append(i.mult)
        data["# Electrons"].append(i.nelectrons)
        data["# atoms"].append(i.natom)
        data["Opt Done"].append(i.optdone)
        data["SCF Energies"].append(i.scfenergies)
        asestart = makease(i.atomcoords[0], i.atomnos, atomcharges=None, atomspins=None, atommasses=None)
        aseend = makease(i.atomcoords[-1], i.atomnos, atomcharges=i.atomcharges['mulliken'], atomspins=None, atommasses=None)
        pymatstart = AseAtomsAdaptor.get_molecule(asestart)
        pymatend = AseAtomsAdaptor.get_molecule(aseend)
        data["Ase Initial"].append(asestart)
        data["Ase Final"].append(aseend)
        data["Pymat Initial"].append(pymatstart)
        data["Pymat Final"].append(pymatend)
        data["Reduced Formula"].append(pymatend.composition.reduced_formula)
        data["Initial Distance Matrix"].append(pymatstart.distance_matrix)
        data["Final Distance Matrix"].append(pymatend.distance_matrix)
        x=PointGroupAnalyzer(pymatstart,tolerance=0.000001)
        data["Initial Symm"].append(x.sch_symbol)
        x=PointGroupAnalyzer(pymatend,tolerance=0.0001)
        data["Final Symm Low Tol"].append(x.sch_symbol)
        x=PointGroupAnalyzer(pymatend,tolerance=0.0000001)
        data["Final Symm High Tol"].append(x.sch_symbol)



    df = pd.DataFrame(data, index=['answer 1', 'answer 2', 'answer 3', 'run 1', 'run 2', 'run 3'])
    
    return df

#display(df)



In [5]:
def SanityFiles(df, runlist, anslist):


    #Sanity checks
    SanScore = 0
    ErrScore = 0
    runcheck = []
    if df.loc["answer 1","Package"]==df.loc["run 1","Package"] and df.loc["answer 2","Package"]==df.loc["run 2","Package"] and df.loc["answer 3","Package"]==df.loc["run 3","Package"]:
        display(Markdown("all calculations used NWchem - 1 mark"))
        SanScore = SanScore + 1
    else:
        display(Markdown("**all calculations did not use Nwchem - you were asked to use Nwchem to perform your calculations**"))

    x = 0

    for i in runlist:
        display(Markdown("for " + i + "\n"))
        if df.loc[anslist[x],"Basis Set"]==df.loc[i,"Basis Set"]:
            display(Markdown("The correct Basis Set was used - 1 mark"))
            SanScore = SanScore + 1  
        else:
            one = df.loc[anslist[x],"Basis Set"]
            two = df.loc[i,"Basis Set"]
            display(Markdown("**An incorrect Basis Set was used, you should have used {one}** **but used {two}** ".format(one=one, two=two)))
            ErrScore = ErrScore + 1
        if df.loc[anslist[x],"Functional"]==df.loc[i,"Functional"]:
            display(Markdown("The correct Functional was used - 1 mark"))
            SanScore = SanScore + 1
        else:
            one = df.loc[anslist[x],"Functional"]
            two = df.loc[i,"Functional"]
            display(Markdown("**An incorrect Functional was used, you should have used {one}** **but used {two}** ".format(one=one, two=two)))
            ErrScore = ErrScore + 1
        if df.loc[anslist[x],"Success"]==df.loc[i,"Success"]:
            display(Markdown("Your calculation completed sucessfully - 1 mark"))
            SanScore = SanScore + 1  
        else:
            display(Markdown("Your calculation did not complete sucessfully - This often means the starting structure was poor and the job did not find an optimal structure in a sensible number of steps"))
            ErrScore = ErrScore + 1
        if df.loc[anslist[x],"Opt Done"]==df.loc[i,"Opt Done"]:
            display(Markdown("A geometry optimisation was performed - 1 mark"))
            SanScore = SanScore + 1  
        else:
            display(Markdown("**A geometry optimisation was not performed - The aim of the exercise is to perform a geometry optimisation to obtain a minium energy structure**"))
            ErrScore = ErrScore + 1
        if df.loc[anslist[x],"# atoms"]==df.loc[i,"# atoms"]:
            display(Markdown("You have the correct number of atoms - 1 mark"))
            SanScore = SanScore + 1  
        else:
            display(Markdown("**You have the incorrect number of atoms - Check the molecule that you've submitted**"))
            ErrScore = ErrScore + 1
        if df.loc[anslist[x],"# Electrons"]==df.loc[i,"# Electrons"]:
            display(Markdown("You have the correct number of electrons - 1 mark"))
            SanScore = SanScore + 1  
        else:
            display(Markdown("**You have the incorrect number of electrons - Check the molecule that you've submitted**"))
            ErrScore = ErrScore + 1
        if df.loc[anslist[x],"Reduced Formula"]==df.loc[i,"Reduced Formula"]:
            display(Markdown("You have the correct number and type of atoms - 1 mark"))
            SanScore = SanScore + 1  
        else:
            display(Markdown("**You have the incorrect number and type of atoms - Check the molecule that you've submitted**"))
            ErrScore = ErrScore + 1
        display(Markdown(""))
        if ErrScore >= 1:
            runcheck.append(1)
        else:
            runcheck.append(0)
        ErrScore = 0
        x = x + 1

    return SanScore, runcheck


In [6]:
def InitialProcess(df, runlist):
    # Intial structures
    InitialScore = 0

    for i in runlist:
        display(Markdown("for " + i + " "))
        p = HungarianOrderMatcher(df.loc["answer 1","Pymat Initial"])
        q = HungarianOrderMatcher(df.loc["answer 1","Pymat Final"])
        fit = p.fit(df.loc[i,"Pymat Initial"])
        fit2 = q.fit(df.loc[i,"Pymat Initial"])

        if fit[1] or fit2[1] <= 1E-2:
            display(Markdown("The starting structure seems sensible - 2 marks \n"))
            InitialScore = InitialScore + 2
        else :
            display(Markdown("**There is a very large error in your starting structure, make sure it makes chemical sense** \n"))


        if df.loc["answer 1","Initial Symm"] == df.loc[i,"Initial Symm"]:
            display(Markdown("Symmetry of starting structure is correct - 2 marks \n"))
            InitialScore = InitialScore + 2
        else:
            display(Markdown("**Symmetry of starting structure is not correct consider a better starting structure** \n"))
            
    return(InitialScore)

In [7]:

def FinalProcess(df, runlist, anslist, runcheck):
    # final Structures
    x = 0
    FinalScore = 0

    for i in runlist:
        if runcheck[x] == 0:
            display(Markdown("for " + i + "\n"))
            EnergyDiff = abs(df.loc[anslist[x],"SCF Energies"][-1] - df.loc[i,"SCF Energies"][-1])
            if EnergyDiff <= 1E-10:
                display(Markdown("Final energy matches within error of the calculation - 10 marks \n"))
                FinalScore = FinalScore + 10
            elif EnergyDiff <= 1E-8:
                display(Markdown("There is a small error in your final energy - 5 marks \n"))
                FinalScore = FinalScore + 5
            elif EnergyDiff <= 1E-5:
                display(Markdown("There is a significant error in your final energy - 3 marks \n"))
                FinalScore = FinalScore + 3
            else :
                display(Markdown("**There is a very large error in your final energy** \n"))

            p = HungarianOrderMatcher(df.loc[anslist[x],"Pymat Final"])
            fit = p.fit(df.loc[i,"Pymat Final"])
            if fit[1] <= 1E-12:
                display(Markdown("The optimised structure is correct - 10 marks \n"))
                FinalScore = FinalScore + 10
            elif fit[1] <= 1E-10:
                display(Markdown("There is a small error in your optimsied structure - 5 marks \n"))
                FinalScore = FinalScore + 5
            elif fit[1] <= 1E-5:
                display(Markdown("There is a significant error in your optimsied structure - 3 marks \n"))
                FinalScore = FinalScore + 3
            else :
                display(Markdown("**There is a very large error in your optimsied structure** \n"))

            bondDiff = np.array(df.loc[anslist[x],"Final Distance Matrix"] - df.loc[i,"Final Distance Matrix"])
            bondDiff = np.linalg.det(bondDiff)
            if abs(bondDiff) < 1E-25:
                display(Markdown("Bond Lengths are correct - 10 marks \n"))
                FinalScore = FinalScore + 10
            elif abs(bondDiff) < 1E-25:
                if bondDiff > 0:
                    display(Markdown("Bond Lengths could be slightly shorter than expected - 5 marks \n"))
                    FinalScore = FinalScore + 5
                else :
                    display(Markdown("Bond Lengths could be slightly longer than expected - 5 marks \n"))
                    FinalScore = FinalScore + 5
            elif abs(bondDiff) < 1E-20:
                if bondDiff > 0:
                    display(Markdown("**Bond Lengths shorter than expected** \n"))
                else :
                    display(Markdown("**Bond Lengths could be longer than expected** \n"))
            else :
                display(Markdown("**Bond Lengths Seem very strange, check the structure** \n"))

            if df.loc[anslist[x],"Final Symm High Tol"] == df.loc[i,"Final Symm High Tol"]:
                display(Markdown("Symmetry of optimised structure is correct - 10 marks \n"))
                FinalScore = FinalScore + 10
            elif df.loc[anslist[x],"Final Symm Low Tol"] == df.loc[i,"Final Symm Low Tol"]:
                display(Markdown("Symmetry of optimised structure is correct to a low tolerance, consider a better starting structure - 5 marks \n"))
                FinalScore = FinalScore + 5
            else:
                display(Markdown("**Symmetry of optimised structure is not correct consider a better starting structure** \n"))            
        else :
            display(Markdown(i + " **failed the sanity check and will not be scored, check errors above to see what went wrong** \n"))
        x = x + 1
    return (FinalScore)

In [8]:
def ExtraProcess(df, runlist):
    # Extra Feedback
    for i in runlist:
        Steps = len(df.loc[i,"SCF Energies"])
        if Steps >= 10:
            display(Markdown("for " + i + " "))
            if True == df.loc[i,"Success"]:
                display(Markdown("your optimisation took a large number of steps to complete, consider a better starting structure"))
            else:
                display(Markdown("your optimisation took a large number of steps and could not find an sensible structure before stopping, consider a better starting structure"))
        elif Steps >= 5:
            display(Markdown("for " + i + " "))
            display(Markdown("your optimisation took a more steps to complete than necessary, consider a better starting structure"))
        else :
            display(Markdown(""))

    if df.loc["run 1","Pymat Final"] == df.loc["run 2","Pymat Initial"] and \
        df.loc["run 2","Pymat Final"] == df.loc["run 3","Pymat Initial"]:
        display(Markdown(""))
    else :
        display(Markdown("When running multiple calculations using different basis sets you can waste alot of time by starting from poor structures \
              instead consider optimsing using a small basis set (This should be quick) and then starting your other runs with larger \
              basis sets from this already optimised structure"))


In [17]:


display(Markdown("# Unit 2 Methane Feedback"))

display(Markdown("Username = {User}".format(User=User)))

display(Markdown("Run 1 = {JobID1} This should be the optimisation of CH<sub>4</sub> using NWchem with 6-31g* and B3LYP".format(JobID1=JobID1)))

display(Markdown("Run 2 = {JobID2} This should be the optimisation of CH<sub>4</sub> using NWchem with 6-3111++g** and B3LYP".format(JobID2=JobID2)))

display(Markdown("Run 3 = {JobID3} This should be the optimisation of CH<sub>4</sub> using NWchem with cc-pVTZ and B3LYP".format(JobID3=JobID3)))
df = ProcessFiles(filelist)

display(Markdown("## Checking the Calculation Used Sensible Settings"))

display(Markdown("In this first section what were checking is that each optimisation was run with sensible settings, that is that the correct software, basis set and functional were used and for each calculation there are the correct number and type of atoms.")) 

display(Markdown("This is essentially a sanity check to make sure the calculations were somewhat sensible and is marked out of a total of 22 marks"))

display(Markdown("Any errors that you should go and look at will be in **bold**. If a calculation fails this sanity check it will not be marked in the final section."))

(SanScore, runcheck) = SanityFiles(df, runlist, anslist)

display(Markdown("## Looking at the Initial Starting Structure for Each Optimisation"))

display(Markdown("In this section I run some basic checks to make sure your starting structure is chemically sensible. The Sanity check has already checked that you have the right number and type of atoms so in this section I compare the starting structure to some other structures from databases or other calculations to make sure the bond lengths/angles make chemical sense.")) 

display(Markdown("These checks may not be perfect but they try and imitate what you should be doing when setting up a calculation. Make sure your starting structure is chemically sensible, if you start from a poor structure the optimisation is going to **a)** struggle to find an mimium energy structure and **b)** if it does it will have taken long to find one, wasting both your time and the computational resources"))
                 
display(Markdown("It then checks to make sure the starting structure has the correct symmetry. This is really important, if a molecule has symmetry the calculation will take this into account and this can significantly speed up the calculation, if going on to use this structure for vibrational spectroscopy calculations, the symmetry also determines this number and IR/Raman activity of the vibrations you calculate."))

display(Markdown("Any errors will be in **bold**."))

InitialScore = InitialProcess(df, runlist)

display(Markdown("## Checking your Optimised Structures"))
display(Markdown("In this section you will marked on the optimised structures you have obtained. Here your submitted structure is compared to a calculation already performed using the same settings. This is looking at the energy, bond lengths and symmerty of your final structure, along with the overall shape."))

display(Markdown("Any calculations that failed the Sanity check above will not be marked. Any Errors will be displayed in **bold** below."))

FinalScore = FinalProcess(df, runlist, anslist, runcheck)


display(Markdown("## Summary of Scores"))

display(Markdown("Your Sanity Score = {SanScore} out of 22".format(SanScore=SanScore)))

display(Markdown("Your Score for Your Initial Structures = {InitialScore} out of 12".format(InitialScore=InitialScore)))

display(Markdown("Your Score for Your Optimised Structures = {FinalScore} out of 120".format(FinalScore=FinalScore)))



display(Markdown("## Extra Guidance and Advice"))

display(Markdown("Finally some additional checks have been run on your calculations to check to see if you could improve how you setup your calculations to speed them up and save you time - particularly when performing calculations on larger molecules in later units."))

display(Markdown("If you calculations pass all the checks there will be not further guidance included"))

ExtraProcess(df, runlist)

# Unit 2 Methane Feedback

Username = eenadbu

Run 1 = 18356 This should be the optimisation of CH<sub>4</sub> using NWchem with 6-31g* and B3LYP

Run 2 = 18746 This should be the optimisation of CH<sub>4</sub> using NWchem with 6-3111++g** and B3LYP

Run 3 = 18777 This should be the optimisation of CH<sub>4</sub> using NWchem with cc-pVTZ and B3LYP

...

Finally some additional checks have been run on your calculations to check to see if you could improve how you setup your calculations to speed them up and save you time - particularly when performing calculations on larger molecules in later units.

If you calculations pass all the checks there will be not further guidance included

for run 2 

your optimisation took a large number of steps to complete, consider a better starting structure

When running multiple calculations using different basis sets you can waste alot of time by starting from poor structures               instead consider optimsing using a small basis set (This should be quick) and then starting your other runs with larger               basis sets from this already optimised structure